In [88]:
# Import blackSheepCPTACmodule and cptac data

import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt, sys
sys.path.insert(0, "/Users/lili/dropbox_lili/common_functions/outlier_analysis/")
import blackSheepCPTACmodule as ol
import importlib
importlib.reload(ol)
import cptac.endometrial as en

In [125]:
# Get data
transcriptomics = en.get_transcriptomics()
clinical = en.get_clinical()
annotations = clinical[['Histologic_Grade_FIGO','Diabetes']]

In [126]:
# Test of the make_outliers_table function to create a table of outliers
# in the transcriptomics data.

outliers = ol.make_outliers_table(transcriptomics, iqrs=2.0, 
                                  up_or_down='up', aggregate=False, 
                                  frac_table=True)

results = ol.compare_groups_outliers(outliers, annotations, frac_filter=.1)
results

# Need to figure out why this is having an AttributeError with tuples

AttributeError: 'tuple' object has no attribute 'transpose'

In [15]:
# This function does what it's supposed to; however,
# it could easily be replaced with something like 
# annotations = clinical[['Histologic_Grade_FIGO', 'Diabetes]]
# Initially I was trying to get this to do the whole process
# interactively, so people would just input values and it would
# work, but I have since split off those parts to try and test them.
# They don't work inside this function (so far), and only some kind of
# worked outside this function. So I simplified. A LOT.

def returnSubsetDF(df):
    print('Here are your options')
    print(set(df))
    boundary_names = []
    print('Please input the columns you would like to binarize. Type "done" to finish.')
    boundary = ''
    while boundary != 'done':
        boundary = input()
        if boundary == 'done':
            break
        elif boundary not in set(df):
            print('Invalid input\nHere are your options:')
            print(set(df))
        else:
            boundary_names.append(boundary)
    new_df = pd.DataFrame()
    for boundary in boundary_names:
        new_df[boundary] = df[boundary]
    return(new_df)

smaller_df = returnSubsetDF(clinical)

Here are your options
{'Tumor_Site_Other', 'BMI', 'Num_full_term_pregnancies', 'Ethnicity', 'Myometrial_invasion_Specify', 'Gender', 'tumor_Stage-Pathological', 'Tumor_Focality', 'Proteomics_Tumor_Normal', 'Patient_ID', 'Tumor_Site', 'Diabetes', 'Country', 'Tumor_purity', 'Race', 'Path_Stage_Reg_Lymph_Nodes-pN', 'LVSI', 'Histologic_type', 'FIGO_stage', 'Tumor_Size_cm', 'Clin_Stage_Dist_Mets-cM', 'Path_Stage_Primary_Tumor-pT', 'Histologic_Grade_FIGO', 'Age', 'Treatment_naive', 'Path_Stage_Dist_Mets-pM'}
Please input the columns you would like to binarize. Type "done" to finish.
FIGO_stage
done


In [23]:
# While this is a great idea and all, I can't figure out how to do this correctly

def binarizeV2(df):
    print('The following columns have been chosen for binarization:\n')
    boundary_names = list(set(df))
    example_dict = {}
    print('Please provide a dictionary to binarize them in the following format:\n')
    for boundary in boundary_names:
          print(boundary)
          example_dict[boundary] = {'option1': ['values', 'that', 'correspond', 'with', 'option1'],
                                   'option2': ['values', 'that', 'correspond', 'with', 'option2'], 
                                    'NaN' : [np.nan]
                                   }
    print(example_dict)
    

def binarizeDictionary(df, boundary_dict):
    boundary_names = list(set(df))
    for boundary in boundary_names:
        for key, value in boundary_dict.items():
            new_col = df[boundary].where(cond=(df[boundary] not in value), other=key)
        new_df[boundary] = new_col
    return(new_df)

my_dictionary = {'Beginning_Stage' : ['IA', 'IB', 'II'], 
                 'Later_Stage' : ['IIIA', 'IIIB', 'IIIC1', 'IIIC2', 'IVB'], 
                 np.nan : [np.nan]
                }
binarizeDictionary(smaller_df, my_dictionary)                
#binarizeV2(smaller_df)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [109]:
binary_columns = new_clinical[['Age', 'Race', 'BMI', 'Diabetes', 'Histologic_Grade_FIGO']]

FIGO grade 2    38
FIGO grade 1    37
FIGO grade 3     8
Name: Histologic_Grade_FIGO, dtype: int64

In [120]:
# Version 5: Probably the best version so far since not everything is done in one function
# Works for Age, Race and BMI as expected. Functions

def binarizeCutOff(my_list, cut_off, replace_low, replace_high):
    return [ replace_low if (x < cut_off) else replace_high if x >= cut_off else x for x in my_list ]

def binarizeRange(my_list, low_bar, high_bar, in_range='In_Range', out_of_range='Out_Of_Range'):
    return [ in_range if (x > low_bar and x < high_bar) else out_of_range  
            if (x < low_bar or x > high_bar) else x for x in my_list ]

'''
I think this binarizeCategorical should work just fine, but for 
whatever reason, it keeps returning 'x' where it should return np.nan
'''

def binarizeCategorical(my_list, option1, option1_list, option2):
    return [ option1 if (x in option1_list) else x if (x == np.nan) else option2 for x in my_list ]
    #return [ option1 if (x in option1_list) else option2 if (x not in option1_list and x != np.nan) else x for x in my_list ]

def binarizeCategoricalV2(my_list, option1, option1_list, option2, option2_list):
    return [ option1 if (x in option1_list) else option2 if (x in option2_list) else x for x in my_list ]

"\nBMI: new_df['BMI'] = binarizeRange(binary_columns['BMI'], 18.5, 2)\nAge: new_df['Age'] = binarizeCutOff(binary_columns['Age'], 50 , 'Young', 'Old')\nRace: new_df['Race'] = binarizeCategorical(binary_columns['Race'], 'European', ['White'], 'Not_European', ['Black or African American', 'Not Reported', 'Asian'])\nDiabetes: new_df['Diabetes'] = binarizeCategorical(binary_columns['Diabetes'], 'Diabetic', ['Yes'], 'Not_Diabetic', ['No'])\nHistologic_Grade_FIGO: new_df['Histologic_Grade_FIGO'] = binarizeCategoricalV2(binary_columns['Histologic_Grade_FIGO'], 'Grade1', ['FIGO Grade 1'], 'Not_Grade1', ['FIGO Grade 2', 'FIGO Grade 3]\n"

In [ ]:
# Example Uses for the functions one at a time
'''
BMI: new_df['BMI'] = binarizeRange(binary_columns['BMI'], 18.5, 2)
Age: new_df['Age'] = binarizeCutOff(binary_columns['Age'], 50 , 'Young', 'Old')
Race: new_df['Race'] = binarizeCategorical(binary_columns['Race'], 'European', ['White'], 'Not_European', ['Black or African American', 'Not Reported', 'Asian'])
Diabetes: new_df['Diabetes'] = binarizeCategorical(binary_columns['Diabetes'], 'Diabetic', ['Yes'], 'Not_Diabetic', ['No'])
Histologic_Grade_FIGO: new_df['Histologic_Grade_FIGO'] = binarizeCategoricalV2(binary_columns['Histologic_Grade_FIGO'], 'Grade1', ['FIGO Grade 1'], 'Not_Grade1', ['FIGO Grade 2', 'FIGO Grade 3]
'''

In [ ]:
# Using the functions
new_df = binary_columns.copy()
new_df = new_df.assign(**{'BMI':binarizeRange(binary_columns['BMI'], 18.5, 25), 
                          'Age':binarizeCutOff(binary_columns['Age'], 50, 'Young', 'Old'), 
                          'Race':binarizeCategoricalV2(binary_columns['Race'], 'European', ['White'], 'Not_European',
                                                       ['Black or African American', 'Not Reported', 'Asian']), 
                          'Diabetes':binarizeCategoricalV2(binary_columns['Diabetes'], 'Diabetic', ['Yes'], 
                                                           'Not_Diabetic', ['No', 'Unknown'])
                         }
                      )
print(new_df)

print(new_df['BMI'].value_counts()) 
print('\n')
print(new_df['Age'].value_counts()) 
print('\n')
print(new_df['Race'].value_counts()) 
print('\n')
print(new_df['Diabetes'].value_counts())

In [ ]:
new_df = new_df.assign(**{'Histologic_Grade_FIGO':binarizeCategoricalV2(binary_columns['Histologic_Grade_FIGO'], 
                                                                        'Grade1', ['FIGO grade 1'], 'Not_Grade1', 
                                                                        ['FIGO grade 2', 'FIGO grade 3'])
                         }
                      )